In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np  
from scipy.optimize import curve_fit
from scipy import stats
from collections import Counter
import random
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']  #This is so that the diagrams are displayed in simplified Chinese

# 1 Data Import

In [ ]:
df_points = pd.read_csv(r'D:\桌面\网络可达性构造文件-最终.csv',encoding='gbk')
df_points.head()

# 2 Network Construction

In [ ]:
# Create a directed graph
G_complex = nx.Graph()

# Add nodes and edges, including weights for nodes and edges
for _, row in df_points.iterrows():
    # Add node, weights come from  (Station Accessibility Weight) and  (Coupled Stations Accessibility) columns
    G_complex.add_node(row['pre_id'], 
                       x=row['lng'], 
                       y=row['lat'], 
                       staion_name=row['pre_station'], 
                       weight1=row['Direction Reachability'],  # The first weight
                       weight2=row['Inter-site Reachability'],        # The second weight, can be modified
                       weight3=row['Off-site Accessibility'],
                       weight4=row['Network Reachability'])        # The third weight, can be modified
                    
    
    G_complex.add_node(row['next_id'], 
                       x=row['next_lng'], 
                       y=row['next_lat'], 
                       staion_name=row['next_station'],
                      weight1=row['next-DR'],  # The first weight
                       weight2=row['next-ISR'],        # The second weight, can be modified
                       weight3=row['next-OSR'],
                       weight4=row['next-NR'])        # The third weight, can be modified 
    
    # Add edge, weights come from 'time' and (Coupled Stations Accessibility) columns
    G_complex.add_edge(row['pre_id'], row['next_id'],
                       line=row['line_name'], 
                       weight=row['length'])

## Equivalent Random Network

In [ ]:
# Get the number of nodes and edges in the original network
num_nodes = G_complex.number_of_nodes()
num_edges = G_complex.number_of_edges()

# Define the results list
results = []

# Repeat 100 operations
for i in range(100):
    # Construct the corresponding random network
    p = num_edges / (num_nodes * (num_nodes - 1) / 2)  # Calculate the probability of connection per pair of nodes
    random_network = nx.erdos_renyi_graph(num_nodes, p)

    # Find the largest connected component
    largest_connected_component = max(nx.connected_components(random_network), key=len)
    G_lcc = random_network.subgraph(largest_connected_component).copy()

    # Calculate the average clustering coefficient of the largest connected component
    average_clustering = nx.average_clustering(G_lcc)

    # Calculate the average path length of the largest connected component
    if nx.is_connected(G_lcc):
        average_path_length = nx.average_shortest_path_length(G_lcc)
    else:
        average_path_length = None  # This should typically not happen as we are calculating from the largest connected component

    # Calculate the degree-degree correlation of the largest connected component
    degree_assortativity = nx.degree_assortativity_coefficient(G_lcc)

    # Store the results in the list
    results.append({
        "Iteration": i + 1,
        "Average Clustering": average_clustering,
        "Average Path Length": average_path_length,
        "Degree Assortativity": degree_assortativity
    })

# Create a DataFrame
results_df = pd.DataFrame(results)

# Output to CSV file
results_df.to_csv("random_network_metrics.csv", index=False)

print("Completed 100 operations and saved the results to the random_network_metrics.csv file.")

完成 100 次操作，并将结果保存到 random_network_metrics.csv 文件中。


# 3 Network Robustness Evaluation

## 3.1 MCSR

In [ ]:
# Assume you already have a complex network G_complex
# #G_complex = nx.Graph()
# Here you need to load or generate your complex network data

# Calculate the number of values for proportion f
nums = 100
# Uniformly generate 100 values between 0 and 1 as proportions f
f_values = np.linspace(0, 1, num=nums)

# Initialize zero arrays of size nums to store the relative sizes of the largest connected subnetwork for each f value
relative_sizes_degree = np.zeros(nums)
relative_sizes_betweenness = np.zeros(nums)
relative_sizes_coreness = np.zeros(nums)
relative_sizes_direct_reachability = np.zeros(nums)
relative_sizes_inter_station_reachability = np.zeros(nums)
relative_sizes_external_reachability = np.zeros(nums)
relative_sizes_random = np.zeros(nums)

# Assign the initial size to the first element of relative sizes
initial_size = len(max(nx.connected_components(G_complex), key=len)) / len(G_complex)
relative_sizes_degree[0] = initial_size
relative_sizes_betweenness[0] = initial_size
relative_sizes_coreness[0] = initial_size
relative_sizes_direct_reachability[0] = initial_size
relative_sizes_inter_station_reachability[0] = initial_size
relative_sizes_external_reachability[0] = initial_size
relative_sizes_random[0] = initial_size

# Create figure and axis objects
fig, ax = plt.subplots(figsize=(10, 6))

# Calculate node removal based on degree
sorted_degrees = sorted(G_complex.degree(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_degrees[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    largest_cc_removed = max(nx.connected_components(G_removed), key=len)
    relative_sizes_degree[i] = len(largest_cc_removed) / len(G_complex)

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_sizes_degree[i], 'ro', markersize=5 * 1.2)

ax.plot(f_values, relative_sizes_degree, 'r-', linewidth=2, label='Based on Degree')

# Calculate node removal based on betweenness
sorted_betweennesses = sorted(nx.betweenness_centrality(G_complex).items(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_betweennesses[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    largest_cc_removed = max(nx.connected_components(G_removed), key=len)
    relative_sizes_betweenness[i] = len(largest_cc_removed) / len(G_complex)

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_sizes_betweenness[i], 'g^', markersize=5 * 1.2)

ax.plot(f_values, relative_sizes_betweenness, 'g-.', linewidth=2, label='Based on Betweenness')

# Calculate node removal based on coreness
coreness = nx.core_number(G_complex)
sorted_coreness = sorted(coreness.items(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_coreness[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    largest_cc_removed = max(nx.connected_components(G_removed), key=len)
    relative_sizes_coreness[i] = len(largest_cc_removed) / len(G_complex)

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_sizes_coreness[i], 'b*', markersize=5 * 1.2)

ax.plot(f_values, relative_sizes_coreness, 'b--', linewidth=2, label='Based on Coreness')

# Calculate node removal based on DR
sorted_direct_reachability = sorted(G_complex.nodes(data='weight1'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_direct_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    largest_cc_removed = max(nx.connected_components(G_removed), key=len)
    relative_sizes_direct_reachability[i] = len(largest_cc_removed) / len(G_complex)

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_sizes_direct_reachability[i], 'c^', markersize=5 * 1.2)

ax.plot(f_values, relative_sizes_direct_reachability, 'c-.', linewidth=2, label='Based on Direct Reachability')

# Calculate node removal based on ISA
sorted_inter_station_reachability = sorted(G_complex.nodes(data='weight2'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_inter_station_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    largest_cc_removed = max(nx.connected_components(G_removed), key=len)
    relative_sizes_inter_station_reachability[i] = len(largest_cc_removed) / len(G_complex)

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_sizes_inter_station_reachability[i], 'm*', markersize=5 * 1.2)

ax.plot(f_values, relative_sizes_inter_station_reachability, 'm--', linewidth=2, label='Based on Inter-Station Reachability')

# Calculate node removal based on OSR
sorted_external_reachability = sorted(G_complex.nodes(data='weight3'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_external_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    largest_cc_removed = max(nx.connected_components(G_removed), key=len)
    relative_sizes_external_reachability[i] = len(largest_cc_removed) / len(G_complex)

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_sizes_external_reachability[i], 'y^', markersize=5 * 1.2)

ax.plot(f_values, relative_sizes_external_reachability, 'y-.', linewidth=2, label='Based on External Reachability')

# Calculate node removal randomly
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = random.sample(list(G_complex.nodes()), num_removed)
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    largest_cc_removed = max(nx.connected_components(G_removed), key=len)
    relative_sizes_random[i] = len(largest_cc_removed) / len(G_complex)

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_sizes_random[i], 'yx', markersize=5 * 1.2)

ax.plot(f_values, relative_sizes_random, 'y:', linewidth=2, label='Random Attack')

# Set figure properties
ax.set_xlabel('Proportion f', fontsize=14)
ax.set_ylabel('Largest Connection Subnetwork Ratio', fontsize=14)
ax.set_title('Robustness Evaluation of Complex Networks', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout()
plt.show()

## 3.2 NE

In [ ]:
# Assume you already have a complex network G_complex
# G_complex = nx.Graph()  # Ensure this is your network graph

# Calculate the number of values for proportion f
nums = 100
# Uniformly generate 100 values between 0 and 1 as proportions f
f_values = np.linspace(0, 1, num=nums)

# Initialize zero arrays of size nums to store the network efficiency for each f value
relative_efficiency_degree = np.zeros(nums)
relative_efficiency_betweenness = np.zeros(nums)
relative_efficiency_coreness = np.zeros(nums)
relative_efficiency_direct_reachability = np.zeros(nums)
relative_efficiency_inter_station_reachability = np.zeros(nums)
relative_efficiency_external_reachability = np.zeros(nums)
relative_efficiency_random = np.zeros(nums)

# Calculate initial network efficiency
initial_efficiency = nx.global_efficiency(G_complex)
relative_efficiency_degree[0] = initial_efficiency
relative_efficiency_betweenness[0] = initial_efficiency
relative_efficiency_coreness[0] = initial_efficiency
relative_efficiency_direct_reachability[0] = initial_efficiency
relative_efficiency_inter_station_reachability[0] = initial_efficiency
relative_efficiency_external_reachability[0] = initial_efficiency
relative_efficiency_random[0] = initial_efficiency

# Create figure and axis objects
fig, ax = plt.subplots(figsize=(10, 6))

# Calculate node removal based on degree
sorted_degrees = sorted(G_complex.degree(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_degrees[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    efficiency_removed = nx.global_efficiency(G_removed)
    relative_efficiency_degree[i] = efficiency_removed

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_efficiency_degree[i], 'ro', markersize=5 * 1.2)

ax.plot(f_values, relative_efficiency_degree, 'r-', linewidth=2, label='Network Efficiency Based on Degree')

# Calculate node removal based on betweenness
sorted_betweennesses = sorted(nx.betweenness_centrality(G_complex).items(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_betweennesses[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    efficiency_removed = nx.global_efficiency(G_removed)
    relative_efficiency_betweenness[i] = efficiency_removed

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_efficiency_betweenness[i], 'g^', markersize=5 * 1.2)

ax.plot(f_values, relative_efficiency_betweenness, 'g-.', linewidth=2, label='Network Efficiency Based on Betweenness')

# Calculate node removal based on coreness
coreness = nx.core_number(G_complex)
sorted_coreness = sorted(coreness.items(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_coreness[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    efficiency_removed = nx.global_efficiency(G_removed)
    relative_efficiency_coreness[i] = efficiency_removed

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_efficiency_coreness[i], 'b*', markersize=5 * 1.2)

ax.plot(f_values, relative_efficiency_coreness, 'b--', linewidth=2, label='Network Efficiency Based on Coreness')

# Calculate node removal based on DR
sorted_direct_reachability = sorted(G_complex.nodes(data='weight1'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_direct_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    efficiency_removed = nx.global_efficiency(G_removed)
    relative_efficiency_direct_reachability[i] = efficiency_removed

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_efficiency_direct_reachability[i], 'c^', markersize=5 * 1.2)

ax.plot(f_values, relative_efficiency_direct_reachability, 'c-.', linewidth=2, label='Network Efficiency Based on Direct Reachability')

# Calculate node removal based on ISA
sorted_inter_station_reachability = sorted(G_complex.nodes(data='weight2'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_inter_station_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    efficiency_removed = nx.global_efficiency(G_removed)
    relative_efficiency_inter_station_reachability[i] = efficiency_removed

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_efficiency_inter_station_reachability[i], 'm*', markersize=5 * 1.2)

ax.plot(f_values, relative_efficiency_inter_station_reachability, 'm--', linewidth=2, label='Network Efficiency Based on Inter-Station Reachability')

# Calculate node removal based on OSR
sorted_external_reachability = sorted(G_complex.nodes(data='weight3'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_external_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    efficiency_removed = nx.global_efficiency(G_removed)
    relative_efficiency_external_reachability[i] = efficiency_removed

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_efficiency_external_reachability[i], 'y^', markersize=5 * 1.2)

ax.plot(f_values, relative_efficiency_external_reachability, 'y-.', linewidth=2, label='Network Efficiency Based on External Reachability')

# Calculate node removal randomly
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = random.sample(list(G_complex.nodes()), num_removed)
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    efficiency_removed = nx.global_efficiency(G_removed)
    relative_efficiency_random[i] = efficiency_removed

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], relative_efficiency_random[i], 'yx', markersize=5 * 1.2)

ax.plot(f_values, relative_efficiency_random, 'y:', linewidth=2, label='Network Efficiency Based on Random Attack')

# Set figure properties
ax.set_xlabel('Proportion f', fontsize=14)
ax.set_ylabel('Network Efficiency', fontsize=14)
ax.set_title('Robustness Evaluation of Complex Networks - Network Efficiency', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout()
plt.show()

## 3.3 NR

In [ ]:
# Assume you already have a complex network G_complex
# G_complex = nx.DiGraph()
# This is where you need to load or generate your complex network data

# Calculate the number of values for proportion f
nums = 100
# Uniformly generate 100 values between 0 and 1 as proportions f
f_values = np.linspace(0, 1, num=nums)

# Initialize zero arrays of size nums to store the global reachability for each f value
global_reachability_degree = np.zeros(nums)
global_reachability_betweenness = np.zeros(nums)
global_reachability_coreness = np.zeros(nums)
global_reachability_coupled_reachability = np.zeros(nums)
global_reachability_random = np.zeros(nums)

# Calculate initial global reachability
initial_global_reachability = sum(nx.get_node_attributes(G_complex, 'weight4').values())
global_reachability_degree[0] = initial_global_reachability
global_reachability_betweenness[0] = initial_global_reachability
global_reachability_coreness[0] = initial_global_reachability  # Coreness initial value
global_reachability_coupled_reachability[0] = initial_global_reachability
global_reachability_random[0] = initial_global_reachability

# Create figure and axis objects
fig, ax = plt.subplots(figsize=(10, 6))

# Node removal based on degree
sorted_degrees = sorted(G_complex.degree(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_degrees[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    global_reachability = sum(nx.get_node_attributes(G_removed, 'weight4').values())
    global_reachability_degree[i] = global_reachability

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], global_reachability_degree[i], 'ro', markersize=5 * 1.2)

ax.plot(f_values, global_reachability_degree, 'r-', linewidth=2, label='Based on Degree')

# Node removal based on betweenness
sorted_betweennesses = sorted(nx.betweenness_centrality(G_complex).items(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_betweennesses[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    global_reachability = sum(nx.get_node_attributes(G_removed, 'weight4').values())
    global_reachability_betweenness[i] = global_reachability

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], global_reachability_betweenness[i], 'g^', markersize=5 * 1.2)

ax.plot(f_values, global_reachability_betweenness, 'g-.', linewidth=2, label='Based on Betweenness')

# Node removal based on coreness
coreness = nx.core_number(G_complex)
sorted_coreness = sorted(coreness.items(), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_coreness[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    global_reachability = sum(nx.get_node_attributes(G_removed, 'weight4').values())
    global_reachability_coreness[i] = global_reachability

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], global_reachability_coreness[i], 'm*', markersize=5 * 1.2)

ax.plot(f_values, global_reachability_coreness, 'm--', linewidth=2, label='Based on Coreness')

# Node removal based on DR
sorted_direct_reachability = sorted(G_complex.nodes(data='weight1'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_direct_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    global_reachability = sum(nx.get_node_attributes(G_removed, 'weight4').values())
    global_reachability_coupled_reachability[i] = global_reachability

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], global_reachability_coupled_reachability[i], 'bs', markersize=5 * 1.2)

ax.plot(f_values, global_reachability_coupled_reachability, 'b-.', linewidth=2, label='Direct Reachability')

# Node removal based on ISA
sorted_inter_station_reachability = sorted(G_complex.nodes(data='weight2'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_inter_station_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    global_reachability = sum(nx.get_node_attributes(G_removed, 'weight4').values())
    global_reachability_coupled_reachability[i] = global_reachability

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], global_reachability_coupled_reachability[i], 'bs', markersize=5 * 1.2)

ax.plot(f_values, global_reachability_coupled_reachability, 'b-.', linewidth=2, label='Inter-Station Reachability')

# Node removal based on OSR
sorted_external_reachability = sorted(G_complex.nodes(data='weight3'), key=lambda x: x[1], reverse=True)
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = [x[0] for x in sorted_external_reachability[:num_removed]]
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    global_reachability = sum(nx.get_node_attributes(G_removed, 'weight4').values())
    global_reachability_coupled_reachability[i] = global_reachability

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], global_reachability_coupled_reachability[i], 'c^', markersize=5 * 1.2)

ax.plot(f_values, global_reachability_coupled_reachability, 'c-.', linewidth=2, label='External Reachability')

# Node removal randomly
for i in range(1, nums):
    num_removed = int(f_values[i] * len(G_complex))
    removed_nodes = random.sample(list(G_complex.nodes()), num_removed)
    G_removed = G_complex.copy()
    G_removed.remove_nodes_from(removed_nodes)
    if len(G_removed) == 0:
        break
    global_reachability = sum(nx.get_node_attributes(G_removed, 'weight4').values())
    global_reachability_random[i] = global_reachability

    # Add a point style every time 4% of nodes are attacked
    if i % 4 == 0:
        ax.plot(f_values[i], global_reachability_random[i], 'yx', markersize=5 * 1.2)

ax.plot(f_values, global_reachability_random, 'y:', linewidth=2, label='Random Attack')

# Set figure properties
ax.set_xlabel('Proportion f', fontsize=14)
ax.set_ylabel('Network Reachability', fontsize=14)
ax.set_title('Evaluation of Network Robustness - Network Reachability', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout()
plt.show()